In [ ]:
!pip install ccxt mpl_finance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 7.1 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.4/291.4 KB 21.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import ccxt
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_finance import candlestick_ochl
import matplotlib.dates as mpdates
import datetime

In [ ]:
class Crypto:
    
#Two instances : ticker and exchange: string  
#Instances associated with class : passed when we initialize/call the call
#Instances are defined in init : first parameter is self
#self: any variable associated with init gets associated with Crypto class
#self.ticker: value associated with ticker gets associated with Crypto class
#    def __init__(self,ticker,xch):
    def __init__(self,xch):
        
#        self.ticker = ticker
#Creating the ticker dynamic/user defined by defining it within method extractData 
        self.xch = xch
#self is written in the method parameters as this method will be initialized when we iniliaze the class

# Method to get OHLCV Data   
    def extractData(self, ticker):
        if self.xch != 'binance':
            print("Unknown Cryptocurrency, OHLCV data will be provided only for Binance")
            return None
    
        exchange = ccxt.binance()
        if exchange.has["fetchOHLCV"]:
            since = exchange.parse8601("2020-11-01T00:00:00Z")
        
            all_orders = []
            while since < exchange.milliseconds():
                limit = 25
                ohlcv = exchange.fetch_ohlcv(symbol = ticker,since=since, timeframe = '1d', limit = limit)
                if len(ohlcv) > 0:
                    since = ohlcv[len(ohlcv)-1][0] + 1
                    all_orders += ohlcv
                else:
                    break
    
        return all_orders
#Method to represent time    
    def cleanData(self,ohlcv_list):
        temp = []
        for i in ohlcv_list:
            timestamp_with_ms = i[0]
            dt = datetime.datetime.fromtimestamp(timestamp_with_ms/1000)
            formatted_time = dt.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            i[0] = dt
            temp.append(i)
        return temp
        
#Methods defined inside class should always have self as first argument
#  Method to Create Dataframe  
    def createDataFrame(self,ohlcv_list):
        df = pd.DataFrame(ohlcv_list,columns = ["date","opening_price","high_price","low_price","closing_price","volume"])
        print(df.head())
        return df

In [ ]:
ohlcv_cc = Crypto(xch = "binance")

In [ ]:
ohlcv_data = ohlcv_cc.extractData(ticker = "BTC/USDT")

In [ ]:
len(ohlcv_data)

In [ ]:
ohlcv_df = ohlcv_cc.createDataFrame(ohlcv_list = ohlcv_data)
ohlcv_df.head()

In [ ]:
class PlotGraph:
    def __init__(self,df):
        self.df = df
        self.df['ewm20'] = df['closing_price'].ewm(span = 20, min_periods = 0,adjust = False,
                                                   ignore_na = False).mean()
        
#  Method to Plot Line Chart of Price vs Exponentially weighted Moving Average for 20 days                                                 
    def createLineChart(self):
        plt.figure(figsize=(16,8))
        plt.plot(self.df["date"],self.df["closing_price"], color = "black", label = "closing_price")
        plt.plot(self.df["date"],self.df["ewm20"], color = "teal", label = "EWMA 20")
        
        plt.title("Closing price of Bitcoin vs EWMA 20")
        plt.legend()
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.show()
        
        return None
    
#  Method to Plot Candlesticks 
    def plotCandlesticks(self):
        df_cs = df.copy()
        try:
            df_cs["date"] = df_cs["date"].map(mpdates.date2num)
        except:
            pass
        #creating subplots
        fig,ax = plt.subplots()
        fig.set_figwidth(20)
        fig.set_figheight(10)
        
        #Plotting the OHLCV data on candlesticks
        candlestick_ohlc(ax,df_cs[["date","opening_price","high_price","low_price","closing_price","volume"]].values,
                         width= 0.5,colorup="green",colordown="red",alpha=0.8)
        plt.plot(df["date"],df["ewm20"], color = "teal",label = "EWMA 20")
        #Setting labels for x axis and y axis
        ax.set_xlabel("Date")
        ax.set_ylabel("Price")
        
        #Setting title
        plt.title("Bitcoin Candlesticks")
        
        #Formatting the Date
        date_format = mpdates.DateFormatter("%d-%m-%Y")
        ax.xaxis.set_major_formatter(date_format)
        fig.autofmt_xdate()
        
        fig.tight_layout()
        
        plt.show()
        
        return None

In [ ]:
plotgraph = PlotGraph(df = ohlcv_df)

In [ ]:
plotgraph.createLineChart()

In [ ]:
plotgraph.plotCandlesticks()